#Global Climate Trends
##Analysis

In [36]:
import pandas as pd
import os
import csv

In [37]:
# Path to folders
input_folder_path = os.path.join(".", "input_data")
output_folder_path = os.path.join(".", "output_data")

# Path to input files
pec_input_file_path = os.path.join(input_folder_path, "primary_energy_consumption.csv")
ecc_input_file_path = os.path.join(input_folder_path, "energy_consumption_per_capita.csv")
ece_input_file_path = os.path.join(input_folder_path, "Energy CO2 Emissions.csv")
# # Path to output files
# pec_output_file_path = os.path.join(output_folder_path, "primary_energy_consumption_by_year_n_country.csv")
# ecc_output_file_path = os.path.join(output_folder_path, "energy_consumption_per_capita_by_year_n_country.csv")
# ece_output_file_path = os.path.join(output_folder_path, "Energy CO2 Emissions_by_year_n_country.csv")

In [38]:
# Read input files
df_pec = pd.read_csv(pec_input_file_path)
df_ecc = pd.read_csv(ecc_input_file_path)
df_ece = pd.read_csv(ece_input_file_path)

In [40]:
print(df_pec.columns)
print(df_ecc.columns)

Index(['Year', 'Country', 'Primary Energy Consumption'], dtype='object')
Index(['Year', 'Country', 'Energy Consumption per Capita'], dtype='object')


In [44]:
merged_df = pd.merge(df_pec, df_ecc, on= ['Year','Country'], how='inner')
# merged_df
# print(df_ece.columns)

merge_df2 = pd.merge(merged_df, df_ece, on= ['Year','Country'], how='inner')
print(merge_df2)

Index(['Carbon Dioxide Emissions from Energy', 'Unnamed: 1', 'Unnamed: 2',
       'Unnamed: 3', 'Unnamed: 4', 'Unnamed: 5', 'Unnamed: 6', 'Unnamed: 7',
       'Unnamed: 8', 'Contents', 'Unnamed: 10', 'Unnamed: 11', 'Unnamed: 12',
       'Unnamed: 13', 'Unnamed: 14', 'Unnamed: 15', 'Unnamed: 16',
       'Unnamed: 17', 'Unnamed: 18', 'Unnamed: 19', 'Unnamed: 20',
       'Unnamed: 21', 'Unnamed: 22', 'Unnamed: 23', 'Unnamed: 24',
       'Unnamed: 25', 'Unnamed: 26', 'Unnamed: 27', 'Unnamed: 28',
       'Unnamed: 29', 'Unnamed: 30', 'Unnamed: 31', 'Unnamed: 32',
       'Unnamed: 33', 'Unnamed: 34', 'Unnamed: 35', 'Unnamed: 36',
       'Unnamed: 37', 'Unnamed: 38', 'Unnamed: 39', 'Unnamed: 40',
       'Unnamed: 41', 'Unnamed: 42', 'Unnamed: 43', 'Unnamed: 44',
       'Unnamed: 45', 'Unnamed: 46', 'Unnamed: 47', 'Unnamed: 48',
       'Unnamed: 49', 'Unnamed: 50', 'Unnamed: 51', 'Unnamed: 52',
       'Unnamed: 53', 'Unnamed: 54', 'Unnamed: 55', 'Unnamed: 56',
       'Unnamed: 57', 'Unnamed: 5

KeyError: 'Year'

In [24]:
# def process_file(filename, columname):
#     input_file_path = os.path.join(input_folder_path, filename)
#     output_file_path = os.path.join(input_folder_path, filename)

#     # Read data and store it in a dataframe
#     data_df = pd.read_csv(input_file_path)

#     df_array = []

#     # Loop over the columns of the dataframe skipping the first column
#     for column in data_df.columns[1:]:
#         # Create a new dataframe with the country and the year
#         cy_list = data_df[['Country', column]].copy()

#         # Adding a new column to the dataframe and positionong
#         cy_list.insert(0, 'Year', column)

#         #Rename for dataset
#         cy_list.rename(columns={column: columname}, inplace=True)

#         # Adding the new dataframe to the array
#         df_array.append(cy_list)

#     return df_array
#     # # Merge dfs 
#     # merged_df = pd.concat(df_array, ignore_index=True)

#     # #Save csv
#     # merged_df.to_csv(output_file_path, index=False, header=True)
#     # return merged_df

In [25]:
# pec_df = process_file("primary_energy_consumption.csv", "Primary Energy Consumption")
# print(pec_df)

[         Year Primary Energy Consumption Primary Energy Consumption
0     Country                     Canada                     Canada
1     Country                     Mexico                     Mexico
2     Country                         US                         US
3     Country                  Argentina                  Argentina
4     Country                     Brazil                     Brazil
...       ...                        ...                        ...
3453  Country                  Sri Lanka                  Sri Lanka
3454  Country                     Taiwan                     Taiwan
3455  Country                   Thailand                   Thailand
3456  Country                    Vietnam                    Vietnam
3457  Country         Other Asia Pacific         Other Asia Pacific

[3458 rows x 3 columns],                             Year             Country  \
0     Primary Energy Consumption              Canada   
1     Primary Energy Consumption             

Energy Consumption Per Capita. DF

In [12]:
# ecpc_df = process_file("energy_consumption_per_capita.csv", "Energy Consumption per Capita")
# ecpc_df